# Homework 2

***

> **a)** Numerically implement the discretiation of $\phi$ in terms of spherical harmonics.

> **b)** Use an initial condition that is peaked around the North Pole, i.e., that looks similar to a Gaussian with a width equal to 0.2. (The exact initial condition does not matter).

> **c)** Evolve the system in time to see from $t=0$ to $t=10$ using your favorite ODE integrator. The resulting evolution should look similar to water waves moving on the surface of a pond, except that the pond is the surface of a sphere.

> **d)** Create a series of figures or a movie that shows how the solution $\phi$ evolves in time. Perform the simulation three times with different choices of $l_{max}$, and at least one of these with a small $l_{max}$ (e.g., $l_{max}=4$) to study the influence of the cut-off $l_{max}$.